In [7]:
# ===========================================================
# DirectML Tabular GAN — GPU-Resident (manual batching)
# ===========================================================
import warnings
warnings.filterwarnings("ignore")

# ---- Imports
import random, numpy as np, pandas as pd
import torch, torch.nn as nn, torch.optim as optim
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import torch_directml as dml

# ---------------------------
# 0) Reproducibility
# ---------------------------
SEED = 999
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

# ---------------------------
# 1) Device (prefer dGPU:1)
# ---------------------------
try:
    DEVICE = dml.device(1)      # your dedicated GPU index; change if needed
    _ = torch.ones(1, device=DEVICE)
except Exception:
    # fallback: try 0, else CPU
    try:
        DEVICE = dml.device(0); _ = torch.ones(1, device=DEVICE)
    except Exception:
        DEVICE = "cpu"
print("Using DEVICE:", DEVICE)

# --------------------------------------------
# 2) Preprocessing helpers + Dataset (inverse)
# --------------------------------------------
def cap_rare_inplace(X: pd.DataFrame, cols, min_count=20, min_frac=None):
    n = len(X)
    for col in cols:
        s = X[col].astype(str)
        t = min_count if min_frac is None else max(min_count, int(min_frac * n))
        vc = s.value_counts(dropna=False)
        rare = vc[vc < t].index
        X.loc[:, col] = s.where(~s.isin(rare), "_OTHER_")

class AutoTabularDataset:
    """Simple holder with fitted preprocessor + tensors."""
    def __init__(self, dataframe: pd.DataFrame, target: str | None = None,
                 min_count=100, min_frac=0.01):
        df = dataframe.copy()

        # Drop obvious IDs
        id_like = [c for c in df.columns
                   if c.lower().endswith("id") or c.lower().endswith("_id") or c.lower()=="id"]
        if id_like: df = df.drop(columns=id_like)

        # X/y split
        if target is not None and target in df.columns:
            y_raw = df[target]; X_raw = df.drop(columns=[target]).copy()
        else:
            y_raw = None; X_raw = df.copy()

        # Types
        self.num_cols = X_raw.select_dtypes(include=[np.number]).columns.tolist()
        self.cat_cols = [c for c in X_raw.columns if c not in self.num_cols]

        # Rare-cap
        if self.cat_cols:
            cap_rare_inplace(X_raw, self.cat_cols, min_count=min_count, min_frac=min_frac)

        # Pipelines
        num_pipe = Pipeline([
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", MinMaxScaler(feature_range=(-1, 1))),
        ])
        try:
            cat_pipe = Pipeline([
                ("imputer", SimpleImputer(strategy="most_frequent")),
                ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False, dtype=np.float32)),
            ])
        except TypeError:
            cat_pipe = Pipeline([
                ("imputer", SimpleImputer(strategy="most_frequent")),
                ("onehot", OneHotEncoder(handle_unknown="ignore", sparse=False, dtype=np.float32)),
            ])

        self.pre = ColumnTransformer(
            transformers=[("num", num_pipe, self.num_cols),
                          ("cat", cat_pipe, self.cat_cols)],
            remainder="drop", verbose_feature_names_out=False
        )

        Xp = self.pre.fit_transform(X_raw).astype(np.float32)
        self.X = torch.as_tensor(Xp, dtype=torch.float32, device="cpu")

        # One-hot group sizes (for inverse)
        self.cat_group_sizes = []
        if self.cat_cols:
            oh: OneHotEncoder = self.pre.named_transformers_["cat"].named_steps["onehot"]
            for cats in oh.categories_:
                self.cat_group_sizes.append(len(cats))

    def _harden_onehots(self, X_fake_np: np.ndarray) -> np.ndarray:
        if not self.cat_cols: return X_fake_np
        out = X_fake_np.copy()
        num_dim = len(self.num_cols)
        start = num_dim
        for g in self.cat_group_sizes:
            if g <= 0: continue
            block = out[:, start:start+g]
            idx = np.argmax(block, axis=1)
            block[:] = 0.0
            block[np.arange(block.shape[0]), idx] = 1.0
            out[:, start:start+g] = block
            start += g
        return out

    def inverse_to_dataframe(self, X_fake: torch.Tensor) -> pd.DataFrame:
        Xf = X_fake.detach().to("cpu").numpy().astype(np.float32)
        if self.num_cols:
            num_dim = len(self.num_cols)
            Xf[:, :num_dim] = np.clip(Xf[:, :num_dim], -1.0, 1.0)
        Xf = self._harden_onehots(Xf)
        try:
            X_inv = self.pre.inverse_transform(Xf)
            cols = self.num_cols + self.cat_cols
            return pd.DataFrame(X_inv, columns=cols)
        except Exception:
            cols = [f"f{i}" for i in range(Xf.shape[1])]
            return pd.DataFrame(Xf, columns=cols)

# ------------------------
# 3) Load data
# ------------------------
FILE_PATH = "../data/Loan_default.csv"
df = pd.read_csv(FILE_PATH)
dataset = AutoTabularDataset(df, target=None, min_count=100, min_frac=0.01)

# ----------------------------------------------------
# 4) Move the WHOLE dataset to GPU once (big speedup)
# ----------------------------------------------------
X_device = dataset.X.to(DEVICE, non_blocking=True)  # one-time host→device copy
N, in_dim = X_device.shape
latent_dim = 64

# ------------------------
# 5) Models (heavier nets)
# ------------------------
width = 2048  # raise/lower based on VRAM

class Generator(nn.Module):
    def __init__(self, z, d):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(z, width), nn.LeakyReLU(0.2),
            nn.Linear(width, width), nn.LeakyReLU(0.2),
            nn.Linear(width, width), nn.LeakyReLU(0.2),
            nn.Linear(width, width), nn.LeakyReLU(0.2),
            nn.Linear(width, d), nn.Tanh(),
        )
    def forward(self, z): return self.net(z)

class Discriminator(nn.Module):
    def __init__(self, d):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d, width), nn.LeakyReLU(0.2),
            nn.Linear(width, width), nn.LeakyReLU(0.2),
            nn.Linear(width, width), nn.LeakyReLU(0.2),
            nn.Linear(width, width), nn.LeakyReLU(0.2),
            nn.Linear(width, 1),
        )
    def forward(self, x): return self.net(x)

G = Generator(latent_dim, in_dim).to(DEVICE)
D = Discriminator(in_dim).to(DEVICE)

# --------------------------------
# 6) Loss, optimizers, (optional EMA)
# --------------------------------
criterion = nn.MSELoss()  # LSGAN: real=1, fake=0
optD = optim.Adam(D.parameters(), lr=3e-4, betas=(0.0, 0.99))
optG = optim.Adam(G.parameters(), lr=1e-4, betas=(0.0, 0.99))

class EMA:
    def __init__(self, model: nn.Module, decay: float = 0.999):
        self.decay = decay
        self.shadow = {k: v.detach().clone() for k, v in model.state_dict().items()}
        self.keys = list(self.shadow.keys()); self.model = model
    @torch.no_grad()
    def update(self):
        msd = self.model.state_dict()
        for k in self.keys:
            self.shadow[k].lerp_(msd[k].detach(), 1.0 - self.decay)
    def copy_to(self, model: nn.Module): model.load_state_dict(self.shadow, strict=False)

ema = EMA(G, decay=0.999)

# -----------------------------
# 7) GPU-resident training loop
# -----------------------------
batch_size = 4096   # try 2048 if it fits; reduce on OOM
num_epochs = 30

# Preallocate labels max size (slice each step)
ones_full  = torch.ones(batch_size, 1, device=DEVICE)
zeros_full = torch.zeros(batch_size, 1, device=DEVICE)

G.train(); D.train()
torch.set_grad_enabled(True)     # global switch back ON
for epoch in range(num_epochs):
    # shuffle indices ON GPU
    perm = torch.randperm(N, device=DEVICE)
    for i in range(0, N, batch_size):
        idx = perm[i:i+batch_size]
        real = X_device.index_select(0, idx)
        bsz = real.size(0)

        real_lab = ones_full[:bsz]
        fake_lab = zeros_full[:bsz]

        # -- D step
        optD.zero_grad(set_to_none=True)
        d_real = D(real); loss_real = criterion(d_real, real_lab)

        z = torch.randn(bsz, latent_dim, device=DEVICE)
        fake = G(z)
        d_fake = D(fake.detach()); loss_fake = criterion(d_fake, fake_lab)

        (loss_real + loss_fake).backward()
        optD.step()

        # -- G step
        optG.zero_grad(set_to_none=True)
        g_loss = criterion(D(fake), real_lab)
        g_loss.backward()
        optG.step()
        ema.update()

    print(f"Epoch {epoch+1:3d}: D_real={loss_real.item():.4f}  D_fake={loss_fake.item():.4f}  G={g_loss.item():.4f}")

# -------------------------------
# 8) Sampling + inverse back to DF
# -------------------------------
G_eval = Generator(latent_dim, in_dim).to(DEVICE)
ema.copy_to(G_eval); G_eval.eval()

with torch.no_grad():
    z = torch.randn(1000, latent_dim, device=DEVICE)
    X_fake = G_eval(z)

synthetic_df = dataset.inverse_to_dataframe(X_fake)
synthetic_df.head(100).to_csv("synthetic_preview.csv", index=False)
print("Saved synthetic_preview.csv (first 100 rows)")


Using DEVICE: privateuseone:1
Epoch   1: D_real=0.1169  D_fake=0.0004  G=0.9775
Epoch   2: D_real=0.0329  D_fake=0.3003  G=1.5659
Epoch   3: D_real=0.0288  D_fake=0.0006  G=0.9104
Epoch   4: D_real=0.0540  D_fake=0.0028  G=0.9746
Epoch   5: D_real=0.0545  D_fake=0.0048  G=0.9056
Epoch   6: D_real=0.0353  D_fake=0.0087  G=0.9694
Epoch   7: D_real=0.0359  D_fake=0.0058  G=1.0218
Epoch   8: D_real=0.0483  D_fake=0.1075  G=1.1257
Epoch   9: D_real=0.0414  D_fake=0.0013  G=0.9404
Epoch  10: D_real=0.0463  D_fake=0.0492  G=1.2766
Epoch  11: D_real=0.0609  D_fake=0.0061  G=0.9574
Epoch  12: D_real=0.0528  D_fake=0.2096  G=1.7349
Epoch  13: D_real=0.7190  D_fake=0.0001  G=0.9560
Epoch  14: D_real=0.0523  D_fake=0.0001  G=0.9745
Epoch  15: D_real=0.0935  D_fake=0.0286  G=1.0002
Epoch  16: D_real=0.1058  D_fake=0.0230  G=0.9989
Epoch  17: D_real=0.0409  D_fake=0.0100  G=0.8833
Epoch  18: D_real=0.0543  D_fake=0.0025  G=0.9184
Epoch  19: D_real=0.0520  D_fake=0.0009  G=0.9407
Epoch  20: D_real=0.

In [8]:
# ===============================================================
# DirectML Tabular GAN — GPU-Resident (fast & stable, drop-in)
# ===============================================================
import warnings
warnings.filterwarnings("ignore")

# ---- Imports
import random, numpy as np, pandas as pd
import torch, torch.nn as nn, torch.optim as optim
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import torch_directml as dml

# ---------------------------
# 0) Reproducibility
# ---------------------------
SEED = 999
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

# ---------------------------
# 1) Device (prefer dGPU:1)
# ---------------------------
try:
    DEVICE = dml.device(1)      # your dedicated GPU index; change if needed
    _ = torch.ones(1, device=DEVICE)
except Exception:
    try:
        DEVICE = dml.device(0); _ = torch.ones(1, device=DEVICE)
    except Exception:
        DEVICE = "cpu"
print("Using DEVICE:", DEVICE)

# --------------------------------------------
# 2) Preprocessing helpers + Dataset (inverse)
# --------------------------------------------
def cap_rare_inplace(X: pd.DataFrame, cols, min_count=20, min_frac=None):
    n = len(X)
    for col in cols:
        s = X[col].astype(str)
        t = min_count if min_frac is None else max(min_count, int(min_frac * n))
        vc = s.value_counts(dropna=False)
        rare = vc[vc < t].index
        X.loc[:, col] = s.where(~s.isin(rare), "_OTHER_")

class AutoTabularDataset:
    """Simple holder with fitted preprocessor + tensors (keeps inverse mapping)."""
    def __init__(self, dataframe: pd.DataFrame, target: str | None = None,
                 min_count=100, min_frac=0.01):
        df = dataframe.copy()

        # Drop obvious IDs
        id_like = [c for c in df.columns
                   if c.lower().endswith("id") or c.lower().endswith("_id") or c.lower()=="id"]
        if id_like: df = df.drop(columns=id_like)

        # X/y split (we synthesize features only; y unused)
        if target is not None and target in df.columns:
            X_raw = df.drop(columns=[target]).copy()
        else:
            X_raw = df.copy()

        # Types
        self.num_cols = X_raw.select_dtypes(include=[np.number]).columns.tolist()
        self.cat_cols = [c for c in X_raw.columns if c not in self.num_cols]

        # Rare-cap
        if self.cat_cols:
            cap_rare_inplace(X_raw, self.cat_cols, min_count=min_count, min_frac=min_frac)

        # Pipelines
        num_pipe = Pipeline([
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", MinMaxScaler(feature_range=(-1, 1))),
        ])
        try:
            cat_pipe = Pipeline([
                ("imputer", SimpleImputer(strategy="most_frequent")),
                ("onehot", OneHotEncoder(handle_unknown="ignore", sparse_output=False, dtype=np.float32)),
            ])
        except TypeError:
            cat_pipe = Pipeline([
                ("imputer", SimpleImputer(strategy="most_frequent")),
                ("onehot", OneHotEncoder(handle_unknown="ignore", sparse=False, dtype=np.float32)),
            ])

        self.pre = ColumnTransformer(
            transformers=[("num", num_pipe, self.num_cols),
                          ("cat", cat_pipe, self.cat_cols)],
            remainder="drop", verbose_feature_names_out=False
        )

        Xp = self.pre.fit_transform(X_raw).astype(np.float32)
        self.X = torch.as_tensor(Xp, dtype=torch.float32, device="cpu")

        # One-hot group sizes (for inverse)
        self.cat_group_sizes = []
        if self.cat_cols:
            oh: OneHotEncoder = self.pre.named_transformers_["cat"].named_steps["onehot"]
            for cats in oh.categories_:
                self.cat_group_sizes.append(len(cats))

    def _harden_onehots(self, X_fake_np: np.ndarray) -> np.ndarray:
        if not self.cat_cols: return X_fake_np
        out = X_fake_np.copy()
        num_dim = len(self.num_cols)
        start = num_dim
        for g in self.cat_group_sizes:
            if g <= 0: continue
            block = out[:, start:start+g]
            idx = np.argmax(block, axis=1)
            block[:] = 0.0
            block[np.arange(block.shape[0]), idx] = 1.0
            out[:, start:start+g] = block
            start += g
        return out

    def inverse_to_dataframe(self, X_fake: torch.Tensor) -> pd.DataFrame:
        Xf = X_fake.detach().to("cpu").numpy().astype(np.float32)
        if self.num_cols:
            num_dim = len(self.num_cols)
            Xf[:, :num_dim] = np.clip(Xf[:, :num_dim], -1.0, 1.0)
        Xf = self._harden_onehots(Xf)
        try:
            X_inv = self.pre.inverse_transform(Xf)
            cols = self.num_cols + self.cat_cols
            return pd.DataFrame(X_inv, columns=cols)
        except Exception:
            cols = [f"f{i}" for i in range(Xf.shape[1])]
            return pd.DataFrame(Xf, columns=cols)

# ------------------------
# 3) Load data
# ------------------------
FILE_PATH = "../data/Loan_default.csv"   # adjust as needed
df = pd.read_csv(FILE_PATH)
dataset = AutoTabularDataset(df, target=None, min_count=100, min_frac=0.01)

# ----------------------------------------------------
# 4) Move the WHOLE dataset to GPU once (big speedup)
# ----------------------------------------------------
X_device = dataset.X.to(DEVICE, non_blocking=True)  # one-time host→device copy
N, in_dim = X_device.shape

# ------------------------
# 5) Models (balanced capacity)
# ------------------------
latent_dim = 64
width_G, width_D = 1024, 768   # G slightly stronger than D (helps G learn)

class Generator(nn.Module):
    def __init__(self, z, d, w=width_G):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(z, w), nn.LeakyReLU(0.2),
            nn.Linear(w, w), nn.LeakyReLU(0.2),
            nn.Linear(w, w), nn.LeakyReLU(0.2),  # 3 blocks total
            nn.Linear(w, d), nn.Tanh(),
        )
    def forward(self, z): return self.net(z)

class Discriminator(nn.Module):
    def __init__(self, d, w=width_D):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(d, w), nn.LeakyReLU(0.2),
            nn.Linear(w, w), nn.LeakyReLU(0.2),  # 2–3 is enough; keep faster
            nn.Linear(w, 1),
        )
    def forward(self, x): return self.net(x)

G = Generator(latent_dim, in_dim).to(DEVICE)
D = Discriminator(in_dim).to(DEVICE)

# --------------------------------
# 6) Loss, optimizers, EMA (quality)
# --------------------------------
criterion = nn.MSELoss()  # LSGAN: real≈1, fake≈0
# Flip advantage: let G learn a tad faster than D
optD = optim.Adam(D.parameters(), lr=2e-4, betas=(0.0, 0.99))
optG = optim.Adam(G.parameters(), lr=3e-4, betas=(0.0, 0.99))

class EMA:
    def __init__(self, model: nn.Module, decay: float = 0.999):
        self.decay = decay
        self.shadow = {k: v.detach().clone() for k, v in model.state_dict().items()}
        self.keys = list(self.shadow.keys()); self.model = model
    @torch.no_grad()
    def update(self):
        msd = self.model.state_dict()
        for k in self.keys:
            self.shadow[k].lerp_(msd[k].detach(), 1.0 - self.decay)
    def copy_to(self, model: nn.Module): model.load_state_dict(self.shadow, strict=False)

ema = EMA(G, decay=0.999)

# -----------------------------------------
# 7) Choose MAXIMUM safe batch size (VRAM)
# -----------------------------------------
def find_max_batch(start=1024, limit=8192, step=2):
    b = start
    while b <= limit:
        try:
            b_eff = min(b, X_device.size(0))
            tmp_real  = X_device[:b_eff]
            tmp_ones  = torch.full((b_eff,1), 0.95, device=DEVICE)  # smoothed labels
            tmp_zeros = torch.full((b_eff,1), 0.05, device=DEVICE)
            tmp_z     = torch.randn(b_eff, latent_dim, device=DEVICE)
            D.zero_grad(set_to_none=True); G.zero_grad(set_to_none=True)
            loss = criterion(D(tmp_real), tmp_ones) + criterion(D(G(tmp_z).detach()), tmp_zeros)
            loss.backward()  # probe memory use
            b *= step
        except RuntimeError as e:
            if "out of memory" in str(e).lower():
                # clear best-effort; DirectML doesn't use CUDA cache but keep symmetry
                break
            else:
                raise
    return max(256, b // step)

batch_size = find_max_batch(start=1024, limit=8192)
# Preallocate big buffers (slice per step)
ones_full  = torch.full((batch_size, 1), 0.95, device=DEVICE)
zeros_full = torch.full((batch_size, 1), 0.05, device=DEVICE)
z_buf      = torch.empty(batch_size, latent_dim, device=DEVICE)

# -----------------------------
# 8) GPU-resident training loop
# -----------------------------
num_epochs = 20
n_critic = 2               # 2 D steps per G step (on average) → more stable

G.train(); D.train(); torch.set_grad_enabled(True)
for epoch in range(num_epochs):
    # Instance noise schedule (decays to 0)
    sigma0 = 0.05
    sigma  = max(0.0, sigma0 * (1 - epoch/num_epochs))

    # shuffle indices ON GPU
    perm = torch.randperm(N, device=DEVICE)
    for i in range(0, N, batch_size):
        idx = perm[i:i+batch_size]
        real = X_device.index_select(0, idx)
        bsz  = real.size(0)

        real_lab = ones_full[:bsz]
        fake_lab = zeros_full[:bsz]
        z = z_buf[:bsz]; z.normal_()   # in-place random (no new alloc)

        # --- D step(s): label smoothing + instance noise
        optD.zero_grad(set_to_none=True)
        real_noisy = real + (sigma * torch.randn_like(real) if sigma > 0 else 0.0)
        fake = G(z)
        fake_noisy = fake.detach() + (sigma * torch.randn_like(fake) if sigma > 0 else 0.0)

        d_real = D(real_noisy);  loss_real = criterion(d_real, real_lab)
        d_fake = D(fake_noisy);  loss_fake = criterion(d_fake, fake_lab)
        (loss_real + loss_fake).backward()
        optD.step()

        # --- G step every n_critic
        if ((i // batch_size) % n_critic) == 0:
            optG.zero_grad(set_to_none=True)
            g_loss = criterion(D(fake), real_lab)  # no noise, no detach
            g_loss.backward()
            optG.step()
            ema.update()

    # Epoch summary (kept light to avoid sync)
    print(f"Epoch {epoch+1:3d}: D_real={loss_real.item():.4f}  D_fake={loss_fake.item():.4f}  G={g_loss.item():.4f}  (bs={batch_size})")

# -------------------------------
# 9) Sampling + inverse back to DF
# -------------------------------
G_eval = Generator(latent_dim, in_dim).to(DEVICE)
ema.copy_to(G_eval); G_eval.eval()
with torch.no_grad():
    z = torch.randn(1000, latent_dim, device=DEVICE)
    X_fake = G_eval(z)

synthetic_df = dataset.inverse_to_dataframe(X_fake)
synthetic_df.head(100).to_csv("synthetic_preview.csv", index=False)
print("Saved synthetic_preview.csv (first 100 rows)")


Using DEVICE: privateuseone:1


RuntimeError: m_device->CreateOperator(&opDesc, IID_PPV_ARGS(&op))